In [1]:
import pandas as pd
import bqplot
from bqplot import *
import us
import bqplot.pyplot as plt
from ipywidgets import widgets
import ipywidgets

In [2]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]

fn = 'Data/ufo-scrubbed-geocoded-time-standardized.csv'
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
ufo_date = ufo.set_index("date")
ufo['year'] = ufo_date.index.year
ufo = ufo[ufo['fips'] != -1]

In [3]:
fips_count = ufo.groupby(['fips','year'])['duration_seconds'].count()
fips_sum = ufo.groupby(['fips','year'])['duration_seconds'].sum()
fips = pd.DataFrame()

In [4]:
fips['total_sightings'] = fips_count
fips['duration_seconds'] =fips_sum
fips['state'] = ufo['state']

In [5]:
fips

total_sightings  duration_seconds state
fips year                                         
1    1939                1           300.000   NaN
     1949                2           660.000   NaN
     1951                1             3.000   NaN
     1956                2          3620.000   NaN
     1961                1           240.000   NaN
     1964                1            60.000   NaN
     1965                1            45.000   NaN
     1966                2           190.000   NaN
     1967                4          1740.000   NaN
     1968                5          4446.000   NaN
     1969                2           420.000   NaN
     1970                1           120.000   NaN
     1971                3          1620.000   NaN
     1972                4          7695.000   NaN
     1973               11         13860.000   NaN
     1974                5          3190.000   NaN
     1975                5          9600.000   NaN
     1976                2           330.000   NaN
     1977                2          1805.000   NaN
     1978                5          5360.000   NaN
     1979                1           300.000   NaN
     1980                4           240.000   NaN
     1984                2           302.000   NaN
     1985                4          1000.000   NaN
     1986                1            10.000   NaN
     1987                2         74400.000   NaN
     1988                2          6300.000   NaN
     1989                1            60.000   NaN
     1992                3           300.000   NaN
     1993                3          3960.000   NaN
...                    ...               ...   ...
56   2006               10         10659.000   NaN
     2007                8          5445.000   NaN
     2008               11          5406.001   NaN
     2009                5           480.000   NaN
     2010               11          5068.000   NaN
     2011               21         29451.000   NaN
     2012               12         26063.000   NaN
     2013               14          6070.000   NaN
     2014                4          8700.000   NaN
72   1954                1           600.000   NaN
     1965                1           300.000   NaN
     1970                1            60.000   NaN
     1972                1             5.000   NaN
     1973                1            60.000   NaN
     1974                1           900.000   NaN
     1975                1           300.000   NaN
     1980                1           600.000   NaN
     1983                1            60.000   NaN
     1989                1            60.000   NaN
     1990                1           900.000   NaN
     1992                1             2.000   NaN
     1993                2           480.000   NaN
     1995                2         18900.000   NaN
     1997                2           270.000   NaN
     1998                4          1090.000   NaN
     1999                3          2880.000   NaN
     2000                2           128.000   NaN
     2001                2           540.000   NaN
     2002                3            55.000   NaN
     2004                1           900.000   NaN

[2736 rows x 3 columns]

In [6]:
fips = fips.reset_index(level=['fips','year'])

In [7]:
@ipywidgets.interact(field = ["total_sightings", "duration_seconds"], year = (min(fips['year']),max(fips['year']),1))
def make_plot(field, year=max(fips['year'])):
    fips_year = fips[fips['year']==int(year)]
    #fips_select = fips_year['fips'fips_year]
    map_fig = plt.figure("Visualizing ufo", min_width=1300, min_height=800)
    def_tt = Tooltip(fields=['name', 'color'], formats=['', ''], labels=['State', field])
    map_res = plt.geo(map_data=bqplot.topo_load('map_data/USStatesMap.json'),
                      colors = {'default_color':'Black'},
                      scales = {'projection': bqplot.AlbersUSA(),
                                'color': ColorScale(colors=['Red','Gray','DeepSkyBlue'])},
                      color=dict(zip(fips_year['fips'],fips_year[field])),
                      tooltip=def_tt) # 
    
    
    plt.show()

interactive(children=(Dropdown(description='field', options=('total_sightings', 'duration_seconds'), value='total_sightings'), IntSlider(value=2014, description='year', max=2014, min=1910), Output()), _dom_classes=('widget-interact',))